### Importaciones

In [120]:
# Paquetes necesarios para la ejecución del notebook
import win32com.client
import os
import datetime
import pandas as pd
import shutil

### Configuracion

In [121]:
# Constantes usadas en el notebook
MAPI = "MAPI" # Messaging Application Programming Interface
DOT = "."
OUTLOOK = "Outlook"
APPLICATION = "Application"
MAIL_ITEM_CODE = 43

# Diccionarios
outlook_folder_codes = {
    0: 'Calendario',
    1: 'Contactos',
    2: 'Borradores',
    3: 'Diario / Jornal',
    4: 'Notas',
    5: 'Tareas',
    6: 'Bandeja de entrada',
    7: 'Bandeja de salida',
    8: 'Elementos enviados',
    9: 'Elementos eliminados',
    10: 'Bandeja de correo del servidor',
    11: 'Conflictos',
    12: 'Elementos de sincronizacion local',
    13: 'Elementos de sincronizacion (Envio)',
    14: 'Elementos de sincronización (Recibo)',
    15: 'Elementos de sincronización completa',
    16: 'Diario de formularios',
    17: 'Carpeta de búsqueda',
    18: 'Bandeja para reglas cliente',
    19: 'Carpeta de sugerencias de correo',
}

# Variables de configuracion
root_address = r'C:\Informacion\rechazos' # Direcccion de carpeta raiz
test_address = r'\prueba'
backup_address = r'\backup'
transportista = {
    "mail_subject": "Reporte de ordenes de carga diario",         # Nombre del asunto de correo
    "local_file_name": "Cf_programadas_por_transportista.xlsx",   # Nombre del archivo local
    "mail_file_address": "",                                      # Direccion del archivo en el correo
}
ruta = {
    "mail_subject": "Venta perdida diaria por cliente y ruta - acum mes",   # Nombre del asunto de correo
    "local_file_name": "Cf_rech_por_ruta.xlsx",                             # Nombre del archivo local
    "mail_file_name": "",                                                   # Direccion del archivo en el correo
}

### Obtener correos de outlook

In [122]:
# Conectar a Outlook
outlook_folder_code = int(input(f'{" ".join(["(" + str(key) + ": " + value + ")" for key, value in outlook_folder_codes.items()])}'))
outlook = win32com.client.Dispatch(OUTLOOK+DOT+APPLICATION).GetNamespace(MAPI)

outlook_folder = outlook.GetDefaultFolder(outlook_folder_code)
print("Tipo de folder: ", outlook_folder)

Tipo de folder:  Bandeja de entrada


### Guardar archivo de outlook

In [123]:
# Buscar el correo más reciente con archivo Excel
mails = outlook_folder.Items

# Ordenar por fecha descendente
mails.Sort("[ReceivedTime]", True) # (mails) Es un objeto lista

def download_excel_file(mails, mail_subject):
    for mail in mails: # Recorrer todos los correos
        if mail.Class != MAIL_ITEM_CODE: # Asegurar que el correo sea un MailItem (otros pueden ser calendario, alertas, etc)
            continue
        if (mail_subject.lower() in mail.Subject.lower()) & (mail.Attachments.Count > 0):
            for attachment in mail.Attachments:
                if attachment.FileName.endswith((".xlsx", ".xls")):
                    file_name = attachment.FileName
                    file_address = os.path.join(root_address+test_address, file_name)
                    attachment.SaveAsFile(file_address)
                    return file_address            

transportista['mail_file_address'] = download_excel_file(mails, transportista['mail_subject'])
ruta['mail_file_address'] = download_excel_file(mails, ruta['mail_subject'])

print(transportista['mail_file_address'])
print(ruta['mail_file_address'])

C:\Informacion\rechazos\prueba\Reporte de Guias de Distribución por FechaV3_6984_8638071872536108373.xlsx
C:\Informacion\rechazos\prueba\Venta perdida x Cliente y ruta diaria_1489_4260537791185343261.xlsx


### Actualizar archivos fuente

In [ ]:
def update_excel_file(mail_file_address, sheet_name, local_file_name):
    # Leer datos de una hoja específica (opcional)
    df_nuevo = pd.read_excel(mail_file_address, sheet_name=sheet_name) # o usa None para todas las hojas

    # Direccion de archivo local
    local_file_address =  os.path.join(root_address+test_address, local_file_name)

    # Backup del archivo
    local_file__backup_address =  os.path.join(root_address+test_address+backup_address, local_file_name)
    shutil.copy2(local_file_address, local_file__backup_address) # Crear el backup

    # TAREA: AGREGAR CORRECTAMENTE LA INFORMACION NUEVA DEL CORREO AL LOCAL


update_excel_file(transportista['mail_file_address'], "Guias", transportista['local_file_name'])
update_excel_file(ruta['mail_file_address'], "Motivos_VP (clte)", ruta['local_file_name'])

c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
